# Installations and Imports

In [247]:
# Installations of libraries
!pip install PyPDF2
!pip install nltk
!pip install pdfminer
!pip install textblob
!pip install spacy

!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_md

2022-03-18 03:34:04.942111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-18 03:34:04.942164: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[+] Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


2022-03-18 03:34:32.437393: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-03-18 03:34:32.437442: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [82]:
# Imports
from textblob import TextBlob
from collections import Counter
from pathlib import Path

import spacy
from spacy import displacy
import fr_core_news_md

import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer

import os
import en_core_web_sm
import glob
import io
import nltk

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

In [2]:
# nltk download
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Documents/PDFs/Texts to Texts

In [85]:
docu_path = "../res/"

# Path to Generated data

# Dump
dump_path = "../dump/"

# Always the same
dataset_path = "../data/" 

# Dynamic due to filename
classification_path = "../data/per_article_classification/"
entities_path = "../data/per_article_entities_label/"
render_path = "../data/sentence_render_img/"

In [86]:
pdf_files = glob.glob(os.path.join(docu_path, "*.pdf"))
doc_files = glob.glob(os.path.join(docu_path, "*.docx"))
txt_files = glob.glob(os.path.join(docu_path, "*.txt"))
accepted_files = []

accepted_files = [*pdf_files, *doc_files, *txt_files]

# Print Sample File
accepted_files[0]

'../res\\3M Moderna vaccines from US Government arrive from COVAX Facility.pdf'

In [87]:
def generate_dataset_dictionary(files):
    dict = {"Docu_ID": [], "File_Name": []}
    
    for idx, file in enumerate(files):
        split_file_name = file.split("\\")
        file_name = split_file_name[1]

        dict["Docu_ID"].append(file_name[0].upper()+"000"+str(idx))
        dict["File_Name"].append(file_name)
    
    return dict
    

In [88]:
document_dict = generate_dataset_dictionary(accepted_files)

# Initial Dataframe
initial_df = pd.DataFrame(document_dict)
initial_df[0:5]

,Docu_ID,File_Name
0,30000,3M Moderna vaccines from US Government arrive ...
1,A0001,Asia Pacific health and finance ministers stre...
2,B0002,Breastfeeding must continue amidst COVID-19.pdf
3,C0003,Community Innovation to Support Surveillance a...
4,D0004,"DOH, RITM, WHO establish subnational laborator..."


In [89]:
initial_df.to_csv(dataset_path + "docu_dataset.csv", index=False, header=True)

In [90]:
def pdf_to_text_sentences(pdf_file, docu_id):
    inFile = open(pdf_file, 'rb')
    resMgr = PDFResourceManager()
    retData = io.StringIO()
    TxtConverter = TextConverter(resMgr, retData,laparams = LAParams())
    interpreter = PDFPageInterpreter(resMgr, TxtConverter)

    # Process each pages
    for page in PDFPage.get_pages(inFile): 
        interpreter.process_page(page)


    # Write to text temporary file
    txt = retData.getvalue()

    # Acquire sentences
    blob = TextBlob(txt)
    sentences = []

    # Append to sentences whilest removing 'sentence(...)' on each sentences/list elements
    for s in blob.sentences:
         sentences.append(str(s).strip()),

    cleaned_sentences = []

    # Append to cleaned_sentences whilest removing new lines or '\n'
    for x in sentences:
         cleaned_sentences.append(x.replace("\n", " "))

    # Create and Open new temp text file and write cleaned sentences on a per line basis
    with open(dump_path + "txt_" + docu_id + ".txt", 'w', encoding='utf-8') as f: 
         f.write('\n'.join(cleaned_sentences))

In [91]:
for idx in initial_df.index:
    pdf_to_text_sentences(docu_path + str(initial_df["File_Name"][idx]), str(initial_df["Docu_ID"][idx]))

In [92]:
txt_files = glob.glob(os.path.join(dump_path, "*.txt"))

text_dict = generate_dataset_dictionary(txt_files)

text_df = pd.DataFrame(text_dict)
text_df[0:5]

,Docu_ID,File_Name
0,T0000,txt_30000.txt
1,T0001,txt_A0001.txt
2,T0002,txt_B0002.txt
3,T0003,txt_C0003.txt
4,T0004,txt_D0004.txt


In [93]:
text_df.to_csv(dataset_path + "text_dataset.csv", index=False, header=True)

# Classify Text File

In [94]:
def open_file(file):
    # Open text file
    raw = open(file, encoding='utf-8').read()
    
    nlp = en_core_web_sm.load()
    nlp.max_length = 3000000
    
    raw_nlp = nlp(raw)
    
    return raw_nlp

In [95]:
def split_sentences(raw_nlp):
    sentences = [x for x in raw_nlp.sents]
    sentence_index = []
    
    # Contain tokenized data and its corresponding sentence index
    for idx1 in range(len(sentences)):
        temporary_sentence = sentences[idx1]
        temporary_tokens = [x for x in temporary_sentence]
        
        for idx2, val in enumerate(temporary_tokens):
            sentence_index.append(idx1)
    
    return sentence_index

In [96]:
def generate_labels_dataset(raw_nlp, docu_id):
    label = ([(x.text, x.label_) for x in raw_nlp.ents])
    word, label = zip(*label)
    labels_df = pd.DataFrame(zip(word, label), columns=["Word", "Label"])
    
    labels_df.to_csv(entities_path + "ent_" + docu_id + ".csv", index=False, header=True)

In [97]:
# Read from csv
text_df = pd.read_csv(dataset_path + "text_dataset.csv")
text_df[0:2]

,Docu_ID,File_Name
0,T0000,txt_30000.txt
1,T0001,txt_A0001.txt


In [98]:
for idx in text_df.index:
    raw_nlp = open_file(dump_path + str(text_df["File_Name"][idx]))
    sentence_index = split_sentences(raw_nlp)
    generate_labels_dataset(raw_nlp, str(text_df["Docu_ID"][idx]))
    
    ent_type = ([(x, x.pos_, spacy.explain(x.tag_), x.tag_, x.ent_iob_, x.ent_type_) for x in raw_nlp])
    
    # Words and Tags list
    word, pos, e_tags, tags, iob, e_type = zip(*ent_type)
    
    classification_df = pd.DataFrame(zip(sentence_index, word, pos, e_tags, tags, iob, e_type), 
                      columns=["Sentence_Index", "Token", "Pos", "Explained_Tag", "Tag", "iob_Tag", "Entity_Type"])
    
    # Output to csv
    classification_df.to_csv(str(classification_path)  + "classification_" + 
                             str(text_df["Docu_ID"][idx]) + ".csv", index=False, header=True)
    
#     print(df["Docu_ID"][idx])
#     print(df["File_Name"][idx])
    
    

# Generate Visuals per Sentence

In [122]:
nlp = spacy.load("en_core_web_sm")

for idx in text_df.index:
    raw_nlp = open_file(dump_path + str(text_df["File_Name"][idx]))
    
    counter = 0
    sentences = [x.text for x in raw_nlp.sents]
    
    # Only read lines with entities
    for i in sentences:
        sent = nlp(i)
        if sent.ents:
            svg = displacy.render(sent, jupyter=False, style='ent')
            displacy.render(sent, jupyter=True, style='ent')
            
            output_path = Path(render_path + initial_df["File_Name"][idx] + "Sentence_" + str(counter) + ".html")
            output_path.open("w", encoding='utf-8').write(svg)
            counter = counter + 1
        else:
            pass